# Data Cleaning for PD Prediction

In [1]:
# import the necessary packages
import numpy as np
import os
import sys
import time
import random
import gzip
import pandas as pd

# set seed 
random.seed(42) # for reproducibility when sampling from the data
np.random.seed(42) # for reproducibility when sampling from the data

csv_file_path = '../Data/Loan_status_2007-2020Q3.gzip'
# df = pd.read_csv(csv_file_path, skiprows=range(1,2825493), nrows=100000)

In [2]:
# Step 1: Determine the total number of rows in the file
total_rows = sum(1 for row in open(csv_file_path)) - 1  # Minus 1 to exclude the header

# Step 2: Randomly select which rows to skip
n = 50000  # Number of rows to sample
skip_rows = sorted(np.random.choice(range(1, total_rows + 1), total_rows - n, replace=False))

# Step 3: Read the file with the skiprows parameter
df_sampled = pd.read_csv(csv_file_path, skiprows=skip_rows)


C:\Users\Utente\AppData\Local\Temp\ipykernel_9448\2987687080.py:9: DtypeWarning: Columns (117) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sampled = pd.read_csv(csv_file_path, skiprows=skip_rows)


In [3]:
df_sampled.shape

(50000, 142)

In [4]:
df_sampled['loan_status'].value_counts() 

loan_status
Fully Paid                                             25684
Current                                                17537
Charged Off                                             6199
Late (31-120 days)                                       268
In Grace Period                                          161
Late (16-30 days)                                         48
Issued                                                    47
Does not meet the credit policy. Status:Fully Paid        43
Does not meet the credit policy. Status:Charged Off       10
Default                                                    3
Name: count, dtype: int64

In [5]:
# display the first 10 columns by percentage of missing data
missing_percentages = df_sampled.isnull().sum() * 100 / len(df_sampled)
missing_percentages.sort_values(ascending=False).head(10)

hardship_loan_status       95.134
deferral_term              95.118
hardship_status            95.118
hardship_dpd               95.118
hardship_length            95.118
payment_plan_start_date    95.118
hardship_end_date          95.118
hardship_type              95.118
hardship_start_date        95.118
hardship_reason            95.118
dtype: float64

In [6]:
# check missing percentage for mths_since_last_delinq
missing_mths_since_last_delinq = df_sampled['mths_since_last_delinq'].isnull().sum() * 100 / len(df_sampled)
missing_mths_since_last_delinq

52.212

In [7]:
df_sampled.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,37,1069357,15000,15000,15000.0,36 months,7.90%,469.36,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,56,1068409,16000,16000,15950.0,60 months,19.91%,423.11,E,E4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,59,1068945,7100,7100,7100.0,36 months,16.77%,252.33,D,D2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,116,1061814,10000,10000,10000.0,36 months,8.90%,317.54,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,126,1068180,2500,2500,2500.0,36 months,14.27%,85.78,C,C2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


Columns that contain more than 50% missing data are not utilised in this study.

In [8]:
# drop the columns with more than 50% missing values
columns_to_drop = missing_percentages[missing_percentages > 50].index
df = df_sampled.drop(columns=columns_to_drop)
df.shape

(50000, 107)

In [9]:
# check datatype of acc_now_delinq, and pct_tl_nvr_dlq
df[['acc_now_delinq', 'pct_tl_nvr_dlq']].dtypes

acc_now_delinq      int64
pct_tl_nvr_dlq    float64
dtype: object

Categorical columns are selected in this phase.

In [10]:
# treat categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns].head()

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,...,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,debt_settlement_flag
0,36 months,7.90%,A,A4,Gateway Hospice,1 year,RENT,Verified,Dec-2011,Fully Paid,...,445xx,OH,Nov-2004,50.4%,f,Jan-2014,Jun-2018,Individual,N,N
1,60 months,19.91%,E,E4,CA Technologies,7 years,RENT,Verified,Dec-2011,Fully Paid,...,017xx,MA,Dec-1998,75.1%,f,Oct-2013,May-2020,Individual,N,N
2,36 months,16.77%,D,D2,Triple D security,1 year,MORTGAGE,Verified,Dec-2011,Fully Paid,...,787xx,TX,Apr-2007,91.2%,f,Jan-2015,May-2019,Individual,N,N
3,36 months,8.90%,A,A5,Renal Support Network,6 years,RENT,Verified,Dec-2011,Fully Paid,...,900xx,CA,Jan-2000,49.5%,f,Mar-2013,May-2018,Individual,N,N
4,36 months,14.27%,C,C2,United states air force,7 years,RENT,Source Verified,Dec-2011,Fully Paid,...,325xx,FL,Nov-2004,37.7%,f,Feb-2012,Feb-2012,Individual,N,N


In [11]:
# check variance of categorical columns
df[categorical_columns].nunique().sort_values()

pymnt_plan                  1
term                        2
application_type            2
initial_list_status         2
hardship_flag               2
debt_settlement_flag        2
verification_status         3
home_ownership              5
grade                       7
loan_status                10
emp_length                 11
purpose                    14
sub_grade                  35
addr_state                 50
last_credit_pull_d        130
last_pymnt_d              142
issue_d                   157
int_rate                  536
earliest_cr_line          640
zip_code                  864
revol_util               1057
title                    1886
emp_title               22495
url                     50000
dtype: int64

The idea is to reduce model complexity by retaining the categorical variables that do not suffer from high cardinality (many categories).

Purpose and title covney similar information. Purpose however contains less categoris and it is thus preferred. Therefore, we keep only purpose. 

Grade and subgrade are hierarchically related. Grade contains less categories and is therefore perferable as it bears less complexity than Subgrade. Therefore, we will keep only the grade column.

Issue date will be kept as time reference and will later be translated into a date format. 

The categorical variables we keep comprise:
- grade, 
- type of home ownership, 
- interest rate, 
- loan status (target), 
- purpose, 
- term, 
- earliest credit line

In [12]:
cat_columns_to_keep = ['issue_d', 
                       'grade', 
                       'home_ownership', 
                       'loan_status', 
                       'int_rate', 
                       'term', 
                       'purpose', 
                       'earliest_cr_line'
                       ]
cat_columns_to_drop = set(categorical_columns) - set(cat_columns_to_keep)
df.drop(columns=cat_columns_to_drop, inplace=True)
df.shape

(50000, 91)

In [13]:
# treat numeric columns
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# check variance of numeric columns
df[numeric_columns].nunique().sort_values()

policy_code                       1
num_tl_120dpd_2m                  2
num_tl_30dpd                      3
acc_now_delinq                    3
collections_12_mths_ex_med        6
                              ...  
tot_cur_bal                   44199
total_rec_int                 46260
total_pymnt_inv               47483
total_pymnt                   48071
id                            50000
Length: 83, dtype: int64

Policy code is useless. We can drop it. Also id is useless and will be removed. 

Columns regarding settlement produce leakage of future information. Therefore they are not included. 

To drive down the number of predictors further, we base the set of numeric predictors on a previous study on Kaggle: https://www.kaggle.com/code/krishnaraj30/xgboost-loan-defaulters-prediction/notebook#%F0%9F%92%BE-Data-Description

Therefore, the numeric features used include:
- loan amount
- installment
- annual income
- debt to income ratio
- number of credit lines the borrower uses
- number of public record bankruptcies

Delinquency metrics are also vital behavioural data to determine whether the chance of a default are higher or not. Therefore, we will consider also the following predictors:
- the percentage of trades where the borrower was not delinquent (pct_tl_nvr_dlq)

In [14]:
numeric_columns_to_keep = ['loan_amnt', 'installment', 'annual_inc', 'dti', 'total_acc', 'pub_rec_bankruptcies', 'pct_tl_nvr_dlq']
numeric_columns_to_drop = set(numeric_columns) - set(numeric_columns_to_keep)
df.drop(columns=numeric_columns_to_drop, inplace=True)
df.shape

(50000, 15)

In [15]:
df['pub_rec_bankruptcies'].value_counts()

pub_rec_bankruptcies
0.0    44135
1.0     5581
2.0      206
3.0       42
4.0        5
5.0        5
7.0        2
8.0        1
6.0        1
Name: count, dtype: int64

In [16]:
df.to_csv('../Data/Loan_status_Cleaned.csv', index=False)